## 2. The Data:

### 2.1. Origin of the data:

     Los datos proceden de dos funtes:
     
    - OnCourt DataBase
    - GitHub Jeff Sackmann
    
     Estas nos proporcionan datos de los partidos desde hace más de 100 años. Evidentemente las estadisticas deportivas han ido mejorando con el tiempo y las más recientes contienen un mayor nivel de detalle. Este ha sido uno de los motivos por los que hemos elegido datasets a partir de año 2000 para el trabajo. También porque el propio deporte va cambiando con el tiempo (torneos, superficies, jugadores, material, etc.) y eso puede distorsionar el modelo.
     
     Antes de utilizar la información, la contrastamos en otras fuentes, fundamentalmente en la web de la ATP y otras webs de resultados deportivos.

**Bibliografia:**
     
     https://github.com/ISANJUA/TFM_ML4sportsBets
     http://www.oncourt.info/index.html
     http://www.tennis-data.co.uk/
     http://www.tennisabstract.com/ 
     https://es.wikipedia.org/wiki/Glosario_de_t%C3%A9rminos_del_tenis
     https://github.com/JeffSackmann/tennis_atp
     http://www.tennislive.net/atp

### 2.2. Theorical Approach:

#### 2.2.1. Analizamos las variables predictores:
     Cada linea del dataset contiene un partido, cada partido contiene 49 features que describen el torneo, los jugadores y las estadisticas de juego. De cara a modelizar este dataset, tenemos que tener en cuenta que hay variables que no van a poder set utilizadas como variables explicativas, bien porque no esten disponibles antes del partido (ej. numero de juegos) o bien porque sean variables descriptivas (ej. date, name of the winner, etc.).
     Vamos a utilizar modelos de clasificacion en los que el Resultado va a ser 1 gana el jugador A, 0 pierde el jugador A.
     Al tener variables de ambos jugadores podriamos elegir utilizar ambas o hacer una nueva feature con la diferencia entre ellas. Si utilizamos esta segunda visión cualquier partido podría tener un Target de 1 o 0 en función de que jugador se ponga primero al hacer un estadístico, por tanto podemos decir que el modelo es simetrico.
     
     A continuación vamos a analizar los tipos de variables:
   

#### a) Features constantes:
    
    Se mantienen constantes en cualquier partido.
    
    1. Hand: Puede ser R o L. Vamos a convertirla en una variable categorica en la que 1 sera R y 0 sera L
    2. Best of: Numero de sets del partido. Vamos a convertirla en una variable categoriaca en la que 1 sea 3 sets y 0 serán 5 sets.
    3. Height: Puede cambiar en el caso de jugadores muy jovenes, sin embargo, con objeto de simplificar el modelo la mantendremos constante.
    
#### b) Features variables, disponibles en cada partido:

    1. Age: Edad de los jugadores en cada encuentro.
    2. Rank/Rank points: Posición en el ranking ATP y puntos ATP. El orden en la clasificación de los jugadores es el mismo utilizando cualquiera de ellas, sin embargo la diferencia entre dos jugadores consecutivos en el ranking es siempre 1 en el caso de Rank mientras que si nos fijamos en los Rank points, el intervalo siempre será diferente.
    3. Odds: Originalmente el objetivo de este era construir un modelo de ML que optimizara el ROI de nuestras apuestas. Para optimizar este resultado no solo es necesario tener un indice de acierto importante sino ademas tener en cuenta el retorno de cada acierto o perdida en caso de error. Por tanto para llevar a cabo este objetivo, necesitariamos cambiar la funcion de coste de nuestro modelo incluyendo la variable odd. Es decir, la función de coste tiene que maximizar nuestras ganancias.
    
     El objetivo de nuestro algoritmo es minimizar la función de coste al maximo. Si el resultado de la función de coste fuese 0 esto supondría que nuestra hipotesis h(x) obtiene la maxima rentabilidad posible.
    
      Dicho de otra manera, si en un modelo normal la función de coste para minimizar la distancia suele ser el método de los minimos cuadrados, en este caso habría que ponderar esas distancias con la cantidad a recibir o a perder. 
    
    Para minimizar la función de coste podriamos utilizar diferentes metodos:
    - Algoritmo de descenso del gradiente
    - Algoritmos de optimizacion avanzada
    
      Sin embargo dada la complejidad de este asunto, en este trabajo los odds simplemente los consideraremos otras variables predictoras que poder utilizar, siendo el objetivo maximar el % de acierto del modelo.
    
     El origen de datos de los odds es diferente al resto del data set.
    
#### c) Features variables, no disponibles en cada partido (requieren iteraciones):

    1. Average Minutes por juego: Duración media del partido
    2. Average Aces por juego: Nº de Aces por juego
    3. Average Double Faults por juego: Nº de Df por juego
    4. Av First In (w_1stIn/w_svpt): Servicios dentro a la primera
    5. Av First Won (w_1stWon / w_1stIn): Puntos ganados en el primer servicio
    6. Av Second Won (w_2ndWon / (w_svpt - w_1stIn)): Puntos ganados en el segundo servicio
    7. Av Break point won ( w_bpSaved / w_bpFaced): Brek Point ganados

     Hay que tener en cuenta que los partidos tienen una duración bien diferenciada según el torneo. Pueden ser torneos a 5 sets o a 3 sets, por tanto lo que haremos es sacar estos estadisticos por juego.

#### d) Features varibles, no disponibles en cada partido (requieren iteraciones muy complejas)

    1. % Victorias en los últimos 5 partidos
    2. % Victoria Head to Head
    3. % Victorias en los ultimos partidos Head to Head
    4. Surface: No utilizaremos esta variable como tal sino que vamos a poner el % de victorias sobre esa superfice (tendremos que implementar una iteración que busque el % partidos ganados sobre la superficie en que se juegue el partido)
    5. Tourney Level: No utilizaremos esta variable como tal sino que vamos a poner el % de victorias sobre esa superfice (tendremos que implementar una iteración que busque el % partidos ganados sobre la superficie en que se juegue el partido)
    
    Por tanto podríamos llegar a disponer de 18 features para poder modelizar.
    

#### 2.2.2. Data challenge:

   **a) Collinearity:**
   
     Los datos que utilizamos para un modelo pueden generar algunos problemas en este: outliers, correlación de los errores, no linearidad de los datos, etc... En este data set el problema que vamos a encontrar es la colinearidad (dificultad para separar individualmente el efecto de los predictores sobre Y) de la información de los partidos. Esto va a provocar que tengamos que desechar alguans variables predictoras.
    
    
   **b) Funcionamiento del data set e iteración:**
   
     Las variables que vamos a utilizar función por diferencias entre winner y loser. Para generar las nuevas, las iteraciones van a ser más complejas, ya que vamos a tener que buscar, agrupar, sumar, etc... por dos columnas distintas (*winner_name, loser_name* ). Si intentamos crear la variable *últimos 5 partidos ganados*, nuestro algoritomo tiene que buscar al jugador en ambas columnas hasta encontrarlo 5 veces y después sumar el número de veces que aparece como ganador.
    
    
   **c) Complejidad de los datos:**
   
     Para hacer un estudio realmente profundo habría que tener una serie de consideraciones que dificultarían mucho el trabajo:
   
  * Diferente rendimiento en las distintas superficies
  * Diferente rendimiento contra diferentes rivales
  * Diferente rendimiento a lo largo de la carrera deportiva de un jugador
   

     Segmentar los datos según todos estos criterios puede provocar reducciones en el dataset que hagan inviable la creación de estadísticos de peso. ¿Cuantos partidos puede haber jugado Rafa Nadal contra el número 50 del mundo sobre cesped?
   
     En futuras reediciones de este trabajo se irá depurando estos puntos. Si se utilizaran en este trabajo se utilizarían como una feature más y no ponderando todo el data set.
   
   
   **d) Errores en los datos:**
   
     Al cruzar la info de jugarores y odds, encontramos muchos NaNs lo que podría reducir el tamaño del dataset consideramblemente. Más adelante decidiremos si utilizamos esta variable en el modelo.